In [1]:
import logging
from datetime import datetime
from tqdm.auto import tqdm

from importlib import reload

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
log = lambda msg: print(f"{datetime.today()} {msg}")

import pandas as pd
import os

import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('__file__'), '../..'))
sys.path.append(root_dir)


In [2]:
import paramiko

hostname = os.getenv('SSH_HOST')
username = os.getenv('SSH_USERNAME')
password = os.getenv('SSH_PASSWORD')
remote_dir = '/home/graham/audio'

# Create an SSH client
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(hostname, username=username, password=password, compress=True)

# List files in the remote directory
stdin, stdout, stderr = ssh.exec_command(f'ls {remote_dir}')
remote_files = [f for f in stdout.read().decode().splitlines() if f.endswith(".opus")]

sftp = ssh.open_sftp()



2025-05-16 09:54:09,180 - INFO - Connected (version 2.0, client OpenSSH_9.6)
2025-05-16 09:54:09,308 - INFO - Authentication (password) successful!
2025-05-16 09:54:11,635 - INFO - [chan 1] Opened sftp connection (server version 3)


# Tests

In [3]:
import audio
reload(audio)


audio.train_find_audio_events_new_mic(sftp, remote_dir)

recording_20250513_054004.opus: 100%|##########| 463k/463k [00:00<00:00, 1.38MB/s]


Temp filename:  C:\Users\graha\AppData\Local\Temp\tmp7vui1cm0.opus

Read Ogg Opus file
Channels:
   1
Frequency (samples per second):
   48000
Buffer Length (bytes):
   2880000
Original audio data shape: (1440000, 1)
Original audio data type: int16
Original audio data range: -1036 to 1027
Original audio data mean: 0.04660902777777778
Original audio data std dev: 84.36350829866926
Sample rate: 48000 Hz
Original duration: 30.00 seconds


recording_20250513_054835.opus: 100%|##########| 463k/463k [00:00<00:00, 1.67MB/s]


Temp filename:  C:\Users\graha\AppData\Local\Temp\tmp02ajknwg.opus

Read Ogg Opus file
Channels:
   1
Frequency (samples per second):
   48000
Buffer Length (bytes):
   2880000
Original audio data shape: (1440000, 1)
Original audio data type: int16
Original audio data range: -213 to 198
Original audio data mean: -0.03665416666666667
Original audio data std dev: 77.93049916128871
Sample rate: 48000 Hz
Original duration: 30.00 seconds


recording_20250513_054535.opus: 100%|##########| 464k/464k [00:00<00:00, 2.59MB/s]


Temp filename:  C:\Users\graha\AppData\Local\Temp\tmp4_dd_5mr.opus

Read Ogg Opus file
Channels:
   1
Frequency (samples per second):
   48000
Buffer Length (bytes):
   2880000
Original audio data shape: (1440000, 1)
Original audio data type: int16
Original audio data range: -358 to 227
Original audio data mean: 0.026611805555555555
Original audio data std dev: 78.68472959649105
Sample rate: 48000 Hz
Original duration: 30.00 seconds
Best parameters found (but not perfect):
{'amplitude_threshold': 230, 'min_event_duration': 0.05, 'merge_distance': 0.3}
Total error: 1

Events found in each file:
recording_20250513_054004.opus: 7 events
  (minimum required: 2)
recording_20250513_054835.opus: 0 events
  (minimum required: 1)
recording_20250513_054535.opus: 0 events


{'amplitude_threshold': 230, 'min_event_duration': 0.05, 'merge_distance': 0.3}

In [6]:
from tqdm.auto import tqdm
import io
import contextlib
from notebooks.Util.DayAndNightOfFinder import day_and_night_of
reload(audio)

bp = audio.best_silence_detection_params

interesting = [f for f in remote_files if day_and_night_of(f.removeprefix('recording_').removesuffix('.opus')) == '2025-05-12']
for idx, i in tqdm(enumerate(interesting), total=len(interesting)):
    output_buffer = io.StringIO()
    with contextlib.redirect_stdout(output_buffer), contextlib.redirect_stderr(output_buffer):
        print(i)

        copied = audio.copy_audio_file(sftp, remote_dir, i)
        data_array, ogg, temp_filename = audio.load_audio_as_ogg(copied.getvalue())
        events = audio.find_audio_events(data_array, ogg.frequency, i,
                            window_size=bp['window_size'],
                            threshold_multiplier=bp['threshold_multiplier'],
                            min_event_duration=bp['min_event_duration'],
                            merge_distance=bp['merge_distance'])
        remote_path = audio.save_events_metadata(events, i, sftp, remote_dir)
        print(f"Saved metadata to {remote_path}")


  0%|          | 0/1438 [00:00<?, ?it/s]

KeyboardInterrupt: 